In [ ]:
import random
import string

import random
import gensim.downloader as api

# Load the Word2Vec model
w2v_model = api.load('word2vec-google-news-300')

# Access the model's vocabulary
# For gensim versions 4.0.0 and above, use `key_to_index`
vocab_list = list(w2v_model.key_to_index.keys())

# For older gensim versions, you might need to use `vocab` instead:
# vocab_list = list(w2v_model.vocab.keys())

# Generate a 5000 membered array with words from the model's vocabulary
words = [random.choice(vocab_list) for _ in range(5)]
print(words[0])

# Display the first 10 entries to verify
# print(words_from_vocab[:10])


# print(words)

# Display the first 10 entries to verify
# random_texts[:10]

import gensim.downloader as api
from nltk.tokenize import word_tokenize
import numpy as np

!pip install nltk

import nltk
nltk.download('punkt')


# Load pre-trained Word2Vec model
w2v_model = api.load('word2vec-google-news-300')

# Function to convert text to embedding
def text_to_embedding(text, embedding_size=128):
    # Tokenize the text
    tokens = word_tokenize(text)

    # Initialize an empty list to store word vectors
    word_vectors = []

    # Iterate over tokens and get their word vectors
    for token in tokens:
        try:
            # Get the word vector for the token
            word_vector = w2v_model[token]
            word_vectors.append(word_vector)
        except KeyError:
            # Ignore tokens not found in the vocabulary
            pass

    # Average the word vectors to get the text embedding
    if word_vectors:
        text_embedding = np.mean(word_vectors, axis=0)
    else:
        # If no word vectors found, use zeros
        text_embedding = np.zeros((embedding_size,))

    # Pad or truncate the embedding to have the desired size
    if len(text_embedding) < embedding_size:
        text_embedding = np.pad(text_embedding, ((0, embedding_size - len(text_embedding))), mode='constant')
    elif len(text_embedding) > embedding_size:
        text_embedding = text_embedding[:embedding_size]

    # Reshape to match the desired shape (1, 128)
    text_embedding = text_embedding.reshape((1, embedding_size))

    return text_embedding

# Example usage
text = "Your text here"
text_embedding = text_to_embedding(text)
print(text_embedding.shape)  # Output: (1, 128)


# Words to generate embeddings for
# words = ["hello", "world", "suresh", "ramesh", "power", "affirmations", "30days", "meditation", "universe", "faith"]

# Generate embeddings for each word
embeddings = []
for word in words:
    embedding = text_to_embedding(word)
    embeddings.append(embedding)
print(embeddings[0].shape)

# # Convert embeddings to array format
embeddings_array = np.array(embeddings)


# print(embeddings)

# # Convert the list of embeddings into a numpy array
# embeddings_array = np.array(embeddings)

# Save the array to a file
np.save("embeddings_array.npy", embeddings_array)


# Convert embeddings to array format
# text_embeddings = np.concatenate(embeddings, axis=0)


import os
from PIL import Image

from google.colab import drive

# Step 1: Mount Google Drive
drive.mount('/content/drive')

def jpg_to_png(input_folder, output_folder):
    # Create output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Loop through all files in the input folder
    for filename in os.listdir(input_folder):
        if filename.endswith(".jpg"):
            # Open the image file
            with Image.open(os.path.join(input_folder, filename)) as img:
                # Construct the output file path with .png extension
                output_file = os.path.splitext(filename)[0] + ".png"
                # Save the image as PNG format in the output folder
                img.save(os.path.join(output_folder, output_file), "PNG")
                print(f"Converted {filename} to {output_file}")

# Change these paths to your input and output folders
input_folder = '/content/drive/My Drive/Dataset/just/'
output_folder = "/content/drive/My Drive/Dataset/png1/"

jpg_to_png(input_folder, output_folder)


from google.colab import drive
!pip install Pillow

# Step 1: Mount Google Drive
drive.mount('/content/drive')

from PIL import Image
import numpy as np
import cv2
import os

def load_and_preprocess_image(image_path):
    img = cv2.imread(image_path)
    img = cv2.resize(img, (224, 224))  # Resize image
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert to RGB
    img = img.astype('float32') / 255.0  # Normalize pixel values to [0, 1]
    return img

def encode_text_into_image(preprocessed_img, text, output_path):
    encoded_img = Image.fromarray((preprocessed_img * 255).astype('uint8'))  # Convert back to PIL image
    binary_text = ''.join(format(ord(char), '08b') for char in text)
    binary_text += '1111111111111110'  # Delimiter

    data_index = 0
    for row in range(encoded_img.height):
        for col in range(encoded_img.width):
            if data_index < len(binary_text):
                pixel = list(encoded_img.getpixel((col, row)))
                for n in range(3):
                    if data_index < len(binary_text):
                        pixel[n] = pixel[n] & ~1 | int(binary_text[data_index])
                        data_index += 1
                    else:
                        break
                encoded_img.putpixel((col, row), tuple(pixel))

    encoded_img.save(output_path, format='PNG')

def decode_text_from_image(image_path):
    img = Image.open(image_path)
    binary_text = ''
    for row in range(img.height):
        for col in range(img.width):
            pixel = img.getpixel((col, row))
            for n in pixel[:3]:
                binary_text += str(n & 1)

    delimiter_index = binary_text.find('1111111111111110')
    if delimiter_index != -1:
        binary_text = binary_text[:delimiter_index]

    decoded_text = ''.join([chr(int(binary_text[i:i+8], 2)) for i in range(0, len(binary_text), 8)])

    return decoded_text

# Directory paths and words
image_directory = '/content/drive/My Drive/Dataset/png1/'
output_directory = '/content/drive/My Drive/Dataset/embedded1/'
# words = ['Word #1', 'Word #2', ..., 'Word #50']

# Ensure output directory exists
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

# Process each image
for i, image_name in enumerate(os.listdir(image_directory)):
    if image_name.endswith('.png'):
        image_path = os.path.join(image_directory, image_name)
        preprocessed_img = load_and_preprocess_image(image_path)

        output_path = os.path.join(output_directory, f"encoded_{image_name}")
        encode_text_into_image(preprocessed_img, words[i], output_path)

        # Decode for demonstration (optional)
        # decoded_text = decode_text_from_image(output_path)
        # print(f"Decoded text from {output_path}: {decoded_text}")


from tensorflow.keras.layers import Input, Concatenate, Conv2D, Conv2DTranspose, Flatten, Dense, LeakyReLU, ZeroPadding2D, Reshape
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import tensorflow as tf

from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Define the GAN architecture
# Generator
def build_generator(input_shape, embedding_shape):
    input_img = Input(shape=input_shape)  # Input for the original images
    input_embedding = Input(shape=embedding_shape)  # Input for the embeddings

    # Reshape embedding input to match the shape of the original image
    embedding_reshaped = Reshape((1, 1, embedding_shape[-1]))(input_embedding)

    # Pad the embedding to match the spatial dimensions of the image input
    padding_height = input_shape[0] - 1
    padding_width = input_shape[1] - 1
    padding = ZeroPadding2D(padding=((0, padding_height), (0, padding_width)))(embedding_reshaped)

    # Concatenate original image and embedding
    concatenated = Concatenate(axis=-1)([input_img, padding])

    # # Generator network architecture
    # x = Conv2D(64, kernel_size=3, strides=2, padding='same', activation='relu')(concatenated)
    # x = Conv2D(128, kernel_size=3, strides=2, padding='same', activation='relu')(x)
    # x = Conv2DTranspose(128, kernel_size=3, strides=2, padding='same', activation='relu')(x)
    # x = Conv2DTranspose(64, kernel_size=3, strides=2, padding='same', activation='relu')(x)
    # generated_img = Conv2DTranspose(3, kernel_size=3, strides=1, padding='same', activation='sigmoid')(x)  # Output image


    x = Conv2D(64, kernel_size=3, strides=2, padding='same')(concatenated)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = Dropout(0.3)(x)

    x = Conv2D(128, kernel_size=3, strides=2, padding='same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = Dropout(0.3)(x)

    x = Conv2DTranspose(128, kernel_size=3, strides=2, padding='same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)

    x = Conv2DTranspose(64, kernel_size=3, strides=2, padding='same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)

    generated_img = Conv2DTranspose(3, kernel_size=3, strides=1, padding='same', activation='sigmoid')(x)

    # Model
    generator = Model(inputs=[input_img, input_embedding], outputs=generated_img)
    return generator

# Discriminator
def build_discriminator(input_shape, embedding_shape):
    input_img = Input(shape=input_shape)  # Input for images with embedded text
    input_embedding = Input(shape=embedding_shape)  # Input for embeddings

        # Reshape embedding input to match the shape of the original image
    embedding_reshaped = Reshape((1, 1, embedding_shape[-1]))(input_embedding)

    # Pad the embedding to match the spatial dimensions of the image input
    padding_height = input_shape[0] - 1
    padding_width = input_shape[1] - 1
    padding = ZeroPadding2D(padding=((0, padding_height), (0, padding_width)))(embedding_reshaped)

    # Concatenate image and embedding
    concatenated = Concatenate(axis=-1)([input_img, padding])

    x = Conv2D(64, kernel_size=3, strides=2, padding='same')(concatenated)
    x = LeakyReLU(alpha=0.2)(x)
    x = Dropout(0.3)(x)

    x = Conv2D(128, kernel_size=3, strides=2, padding='same')(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = Dropout(0.3)(x)

    # Discriminator network architecture
    # x = Conv2D(64, kernel_size=3, strides=2, padding='same', activation=LeakyReLU(alpha=0.2))(concatenated)
    # x = Conv2D(128, kernel_size=3, strides=2, padding='same', activation=LeakyReLU(alpha=0.2))(x)
    x = Flatten()(x)
    validity = Dense(1, activation='sigmoid')(x)  # Output validity

    # Model
    discriminator = Model(inputs=[input_img, input_embedding], outputs=validity)
    return discriminator

# Adversarial Model
def build_adversarial_model(generator, discriminator, input_shape, embedding_shape):
    input_img = Input(shape=input_shape)  # Input for original images
    input_embedding = Input(shape=embedding_shape)  # Input for embeddings

    # Generate fake images
    generated_img = generator([input_img, input_embedding])

    # Disable discriminator training during generator training
    discriminator.trainable = False

    # Validity of generated images
    validity = discriminator([generated_img, input_embedding])

    # Adversarial model
    adversarial_model = Model(inputs=[input_img, input_embedding], outputs=validity)
    return adversarial_model

embedding_shape = (1, 128)  # Assuming text embeddings are of shape (1, 128)
input_shape = (224, 224, 3)

# Build and compile discriminator
discriminator = build_discriminator(input_shape, embedding_shape)
discriminator.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])

from tensorflow.keras.optimizers.schedules import ExponentialDecay

# Define learning rate schedule
initial_learning_rate = 0.0001
lr_schedule = ExponentialDecay(
    initial_learning_rate=initial_learning_rate,
    decay_steps=1000,
    decay_rate=0.96,
    staircase=True)

# Optimizers with learning rate schedule
optimizer_gen = tf.keras.optimizers.Adam(learning_rate=lr_schedule, beta_1=0.5)
optimizer_disc = tf.keras.optimizers.Adam(learning_rate=lr_schedule, beta_1=0.5)

# Build generator
generator = build_generator(input_shape, embedding_shape)


# Build adversarial model
adversarial_model = build_adversarial_model(generator, discriminator, input_shape, embedding_shape)
adversarial_model.compile(optimizer=optimizer_gen, loss='binary_crossentropy', metrics=['accuracy'])

# Compile discriminator (separately for fine-tuning)
discriminator.compile(optimizer=optimizer_disc, loss='binary_crossentropy', metrics=['accuracy'])

# Compile generator (separately for fine-tuning)
generator.compile(optimizer=optimizer_gen, loss='binary_crossentropy', metrics=['accuracy'])



import os
import cv2
import numpy as np
from sklearn.metrics import confusion_matrix
import tensorflow as tf


# Function to load and preprocess images
def load_and_preprocess_image(image_path):
    img = cv2.imread(image_path)
    img = cv2.resize(img, (224, 224))  # Resize image
    # img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert to RGB
    # img = img.astype('float32') / 255.0  # Normalize pixel values to [0, 1]
    return img

def calculate_precision_recall(conf_matrix):
    # Extract true positives, false positives, and false negatives from the confusion matrix
    TP = conf_matrix[1, 1]
    FP = conf_matrix[0, 1]
    FN = conf_matrix[1, 0]

    # Calculate precision
    precision = TP / (TP + FP)

    # Calculate recall
    recall = TP / (TP + FN)

    return precision, recall

# Load embeddings of real images without hidden content
embeddings_array = np.load('embeddings_array.npy')
print(embeddings_array[0].shape)
# print(embeddings_array)
# Define batch size
batch_size = 1
# print(len(embeddings_array))
# Define number of training steps
num_steps = len(embeddings_array) // batch_size
# print(num_steps)

# Define number of epochs
epochs = 1  # Adjusted number of epochs

# Initialize variables to accumulate predictions and true labels
all_predictions = []
all_true_labels = []

# Training loop
for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")

    # Training loop
    for step in range(num_steps):
        # Prepare batch of real images without hidden content and their corresponding embeddings
        batch_embeddings = embeddings_array[step * batch_size: (step + 1) * batch_size]
        # Adjust embeddings shape
        # batch_embeddings = np.expand_dims(batch_embeddings, axis=1)  # Now has shape (1, 1, 128)
        batch_images = []
        for img_name in os.listdir(input_images)[step * batch_size: (step + 1) * batch_size]:
            img_path = os.path.join(input_images, img_name)
            img = load_and_preprocess_image(img_path)
            batch_images.append(img)
        batch_images = np.array(batch_images)

        # Prepare batch of embedded images and their corresponding embeddings
        batch_embedded_images = []
        for img_name in os.listdir(embedded_images)[step * batch_size: (step + 1) * batch_size]:
            img_path = os.path.join(embedded_images, img_name)
            img = load_and_preprocess_image(img_path)
            # img = cv2.imread(image_path)
            batch_embedded_images.append(img)
        print(batch_embedded_images[0].shape)
        batch_embedded_images = np.array(batch_embedded_images)

        # Train the discriminator
        d_loss_real = discriminator.train_on_batch([batch_embedded_images, batch_embeddings], tf.ones((batch_size,1)))  # Real images should be labeled as 1
        fake_images = generator.predict([batch_images, batch_embeddings])  # Generate fake images using real embeddings
        d_loss_fake = discriminator.train_on_batch([fake_images, batch_embeddings], tf.zeros((batch_size,1)))  # Fake images should be labeled as 0
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        # Train the generator (adversarial model)
        g_loss = adversarial_model.train_on_batch([batch_images, batch_embeddings], np.ones((batch_size, 1)))  # Fool the discriminator

        # Calculate discriminator accuracy
        discriminator_accuracy_real = d_loss_real[1]
        discriminator_accuracy_fake = d_loss_fake[1]
        discriminator_accuracy = 0.5 * (discriminator_accuracy_real + discriminator_accuracy_fake)

        # Calculate generator accuracy
        generator_accuracy = g_loss[1]

        # Print progress
        print(f"Step: {step}/{num_steps}, D_loss: {d_loss}, G_loss: {g_loss}, D_accuracy: {discriminator_accuracy}, G_accuracy: {generator_accuracy}")

        # Get discriminator predictions for real and fake images
        discriminator_predictions_real = discriminator.predict([batch_embedded_images, batch_embeddings])
        discriminator_predictions_fake = discriminator.predict([fake_images, batch_embeddings])

        # Threshold discriminator predictions
        discriminator_predictions_real = (discriminator_predictions_real >= 0.5).astype(int)
        discriminator_predictions_fake = (discriminator_predictions_fake < 0.5).astype(int)

        # Accumulate predictions and true labels
        all_predictions.extend(discriminator_predictions_real)
        all_predictions.extend(discriminator_predictions_fake)
        all_true_labels.extend([1] * batch_size)
        all_true_labels.extend([0] * batch_size)

    conf_matrix = confusion_matrix(all_true_labels, all_predictions)

    # Print confusion matrix
    print("Confusion Matrix:")
    print(conf_matrix)

    # Calculate precision and recall
    precision, recall = calculate_precision_recall(conf_matrix)

    # Print precision and recall
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")

# Save the trained models if needed
discriminator.save('discriminator.h5')
generator.save('generator.h5')


#Decoder architecture




from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout,Reshape
from tensorflow.keras.models import Model

from tensorflow.keras.layers import Input, LSTM, Dense, TimeDistributed, Lambda, RepeatVector
from tensorflow.keras.models import Model
import tensorflow as tf
from tensorflow.keras import backend as K

# Load VGG16 as the base model, excluding the top (classification) layer
base_model = VGG16(include_top=False, weights='imagenet', input_shape=(224, 224, 3))

# Set the layers of the base model to be non-trainable to preserve pre-trained features
for layer in base_model.layers:
    layer.trainable = False

# Add new layers on top of the base model
x = base_model.output
x = GlobalAveragePooling2D()(x)  # Reduces the features to a 1D vector to prevent overfitting

# Optional: Add a Dropout layer for regularization
x = Dropout(0.5)(x)

# Add a Dense layer to allow the network to learn complex patterns
x = Dense(512, activation='relu')(x)

# Final Dense layer with output shape (128,) to match text embeddings
final_output = Dense(128, activation='relu')(x)


# Reshape the output to (None, 1, 128)
final_output_reshaped = Reshape((1, 128))(final_output)

# Define the new model, from base model input to the new output
encoder_model = Model(inputs=base_model.input, outputs=final_output_reshaped)

# Display the model summary to verify the architecture
# encoder_model.summary()


from tensorflow.keras.layers import Input, LSTM, Dense, Embedding, RepeatVector,TimeDistributed
from tensorflow.keras.models import Model

vocab_size = 4500  # Adjust based on your dataset
max_seq_length = 20  # Maximum length of text sequences
embedding_dim = 128  # Dimension of the embeddings

# Decoder Model
decoder_inputs = Input(shape=(1, embedding_dim))  # Adjusted to match embedding shape

# Repeat the encoder's output to match the desired sequence length
# decoder_repeated = RepeatVector(max_seq_length)(decoder_inputs)

# LSTM layer
# decoder_lstm = LSTM(256, return_sequences=True)(decoder_inputs)

# Repeat the input for the sequence length to match the LSTM input requirements
repeated_decoder_inputs = RepeatVector(max_seq_length)(Lambda(lambda x: K.squeeze(x, axis=1))(decoder_inputs))

# LSTM layer
decoder_lstm = LSTM(256, return_sequences=True, name="decoder_lstm")(repeated_decoder_inputs)

# Use TimeDistributed to apply Dense layer to each time step independently
decoder_time_dist = TimeDistributed(Dense(vocab_size, activation='softmax'))(decoder_lstm)

# Building the model with TimeDistributed Dense layer
decoder_model = Model(inputs=decoder_inputs, outputs=decoder_time_dist)

# # Predicting each word in the sequence
# decoder_dense = Dense(vocab_size, activation='softmax')
# decoder_outputs = decoder_dense(decoder_lstm)

# # Building the model
# decoder_model = Model(inputs=decoder_inputs, outputs=decoder_outputs)

# Model compilation
decoder_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Display the model architecture
# decoder_model.summary()


from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input


# Define the full model that chains encoder and decoder
def build_end_to_end_model(encoder, decoder):
    # Define the input for the full model, which is the input for the encoder
    full_model_input = Input(shape=(224, 224, 3), name='full_model_input')

    # Get the encoder output (embeddings) for the given input
    encoder_output = encoder(full_model_input)

    # Feed the encoder output (embeddings) as input to the decoder
    decoder_output = decoder(encoder_output)

    print(decoder_output.shape)

    # Define the full end-to-end model
    full_model = Model(inputs=full_model_input, outputs=decoder_output, name='end_to_end_model')

    return full_model

# Build the full model
end_to_end_model = build_end_to_end_model(encoder_model, decoder_model)

# Model compilation (adjust optimizer and loss as needed)
end_to_end_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Display the model summary to verify the architecture
end_to_end_model.summary()

from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
import numpy as np
import os
import cv2
import tensorflow as tf
from sklearn.metrics import confusion_matrix, precision_score, recall_score


# def preprocess_text(texts, tokenizer, max_seq_length):
#     sequences = tokenizer.texts_to_sequences(texts)
#     padded_sequences = pad_sequences(sequences, maxlen=max_seq_length, padding='post')
#     return padded_sequences

tokenizer = Tokenizer(num_words=vocab_size)
# Fit your tokenizer here on your text data
# print(words)

tokenizer.fit_on_texts(words)

def convert_to_classes(predictions):
    return np.argmax(predictions, axis=-1)

# Initialize lists to store precision and recall
epoch_precisions = []
epoch_recalls = []

text_sequences = words


# Directory containing preprocessed images
image_directory = '/content/drive/My Drive/Dataset/embedded1/'
loaded_images = []

# Iterate through each file in the directory
for filename in os.listdir(image_directory):
    if filename.endswith(".jpg") or filename.endswith(".png"):  # Check for image files
        # Construct the full path to the image file
        file_path = os.path.join(image_directory, filename)

        # Load and preprocess the image
        img = cv2.imread(file_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = img / 255.0  # Normalize image pixels to [0, 1]
        loaded_images.append(img)

images_array = np.array(loaded_images)

text_sequences = tokenizer.texts_to_sequences(text_sequences)
target_data = pad_sequences(text_sequences, maxlen=max_seq_length, padding='post')
num_samples = len(loaded_images)
# One-hot encode your sequences. This might be memory-intensive.
target_data_one_hot = np.zeros((num_samples, max_seq_length, vocab_size), dtype='float32')

for i, seq in enumerate(target_data):
    for j, word_index in enumerate(seq):
        if word_index != 0:  # Assuming 0 is used for padding
            target_data_one_hot[i, j, word_index] = 1.0

# print(images_array.shape)  # Should be (num_samples, 224, 224, 3)
print(target_data_one_hot.shape)  # Should be (num_samples, max_seq_length)



# Train the model

end_to_end_model.fit(images_array, target_data_one_hot, batch_size=32, epochs=10, verbose=1)